In [1]:
!pip install -q -U accelerate bitsandbytes git+https://github.com/huggingface/transformers.git
!pip install datasets -q
!pip install peft -q

In [3]:
from transformers import AutoProcessor, PaliGemmaConfig, PaliGemmaForConditionalGeneration
from PIL import Image
import requests
import torch

In [4]:
model_id = "google/paligemma-3b-mix-224"

In [5]:
config = PaliGemmaConfig.from_pretrained(model_id)
config.hidden_activation = 'gelu_pytorch_tanh'

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

In [6]:
model = PaliGemmaForConditionalGeneration.from_pretrained(model_id, config=config)

model.safetensors.index.json:   0%|          | 0.00/62.6k [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/137 [00:00<?, ?B/s]

In [7]:
processor = AutoProcessor.from_pretrained(model_id, config=config)

preprocessor_config.json:   0%|          | 0.00/699 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/40.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/607 [00:00<?, ?B/s]

In [10]:
# Input 1: The prompt
prompt = "What is on the flower?"

# Input 2: The image
image_url = "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/bee.jpg?download=true"
image = Image.open(requests.get(image_url, stream=True).raw)

# Encode the inputs
inputs = processor(prompt, image, return_tensors="pt")
inputs_length = inputs["input_ids"].shape[-1] 

with torch.inference_mode():
    # Perform the inference
    outputs = model.generate(**inputs, max_new_tokens=20, do_sample=False)
    output = outputs[0][inputs_length:] # filter out the prompt

    # Decode the output
    output_decoded = processor.decode(output, skip_special_tokens=True)
    print(output_decoded)

bee
